# Project 1.

## Deadline: April 23, 22:00 ```Ավելի ուշ ուղարկված աշխատանքներ չեն ընդունվի։```


```
Complete all the tasks in the jupyter notebook and name it name_last-name_project1.ipynb. Send me a zip file called name_last-name_project1 containing 2 files: the csv file with the data that you have scrapped in Task 1 and the jupyter notebook. 
Send the zip file to my email: shahanearushanyan@gmail.com
```

```
Կատարեք բոլոր առաջադրանքները jupyter notebook-ում և անվանեք այն name_last-name_project1.ipynb. Ուղարկեք ինձ zip ֆայլ name_last-name_project1 անունով, որը կպարունակի 2 ֆայլ՝ scrap արված տվյալներով csv ֆայլը և jupyter notebook-ը։ 
Ուղարկեք zip ֆայլն իմ mail-ին՝ shahanearushanyan@gmail.com
```

## Task 1.

```
Scrap Amazon to get data for their best seller books. Get title, author, rating, number of users rated and price information for all available pages of products. (e.g. if there are 10 pages of best seller books, you need to scrap all the pages, not just the first page). Create a pandas dataframe and keep the scrapped information inside the dataframe, in separate columns.
```

```
Scrap արեք Amazon-ի կայքը և ստացեք best seller (լավագույն վաճառվող) գրքերի տվյալները։ Ստացեք վերնագիր, հեղինակ, գնահատական, գնահատող օգտատերերի քանակ և գին բոլոր հասանելի էջերի գրքերի համար։ (Օրինակ՝ եթե կա 10 էջ, ապա պետք է ստանաք բոլոր 10 էջերում տեղ գտած ինֆորմացիան)։ Ստեղծեք pandas dataframe աղյուսակ և այնտեղ պահեք Ձեր ստացած տվյալներն առանձին սյուներով։  
```

In [1]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import re
from Levenshtein import distance as levenshtein_distance

In [2]:
def scrape_data():
    headers = {
            'authority': 'www.amazon.com',
            'pragma': 'no-cache',
            'cache-control': 'no-cache',
            'dnt': '1',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'sec-fetch-site': 'none',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-dest': 'document',
            'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        }

    idx = 1
    books = pd.DataFrame(columns=['Name', 'Price', 'Author', 'Rating', 'Users'])
    i = 0

    while True:
        response = requests.get("https://www.amazon.com/best-sellers-books-Amazon/zgbs/books/ref=zg_bs_pg_{}?_encoding=UTF8&pg={}".format(idx,idx), headers=headers)
        if(response.status_code != 200):
            break
        content = response.content
        soup = BeautifulSoup(content)

        for el in soup.find_all('span', attrs={'class':'aok-inline-block zg-item'}):
            rating = 0
            count = 0
            book_name = str(el.find_all('div', attrs={'class': 'a-section a-spacing-small'})).split('"')[3]
            price = float(el.find('span', attrs={'class': 'p13n-sc-price'}).text.strip().replace('$',''))
            try:
                rating = float(str(el.find('div', attrs={'class':'a-icon-row a-spacing-none'}).find('a', attrs={'class':'a-link-normal'})['title']).split(' ')[0])
                count = int(el.find('a', attrs={'class': 'a-size-small a-link-normal'}).text.strip().replace(',',''))
            except:
                pass
            try:
                author = str(el.find('a', attrs={'class':'a-size-small a-link-child'}).text.strip())
            except:
                author = el.find('span', attrs={'class':'a-size-small a-color-base'}).text.strip()

            books.loc[i] = [book_name, price, author, rating, count]
            i += 1
        idx += 1

    books.to_csv('data.csv')

scrape_data()

## Task 2.

```
Create classes Library and Book using the skeleton below and following function descriptions.
Then use the data you got from Task 1., create an object of type Library and fill it with a list
of objects of type Book. Test all the methods that you have implemented.
```

```
Ստեղծեք Library և Book class-երը՝ օգտագործելով տրված կմախքն ու հետևելով ֆունկցիաների նկարագրություններին։ Այնուհետև օգտագործեք Task 1.-ում ստացված տվյալները, ստեղծեք Library տիպի օբյեկտ լցված գիրք տիպի օբյեկտներով։ Փորձարկեք Ձեր գրած բոլոր ֆունկցիաները։ 
```

In [4]:
class Book:
    
    def __init__(self, title, author, rating, num_users_rated, price):
        """
        Create the attributes title, author, rating, num_users_rated and price
        
        Ստեղծեք title, author, rating, num_users_rated և price attribute-ները
        """
        
        self.title = title
        self.author = author
        self.rating = rating
        self.num_uers_rated = num_users_rated
        self.price = price
            
    def pretty_print(self):
        """
        Print book information in the following format:
        $title by $author with rating $rating costs $price.
        
        Տպեք տվյալ գրքին վերաբերող ինֆորմացիան հետևյալ ֆորմատով՝
        $title by $author with rating $rating costs $price.
        """
        
        print("{} by {} with rating {} costs ${}".format(self.title, self.author, self.rating, self.price))
    
    def rating_eval(self):
        """
        Evaluates the statistical significance of the rating. The method should return 
        "statistically significant rating" (or ssr) in case more than 5000 users have
        rated the following book and should return "not statistically significant rating" (or nssr), otherwise.
        
        Գնահատեք թե որքանով ստատիստիկորեն արժեքավոր է գրքին տրված գնահատականը։ Ֆունկցիան պետք է վերադարձնի 
        "statistically significant rating" (կամ ssr), եթե գիրքը գնահատել է ավելի քան 5000 օգտատեր և  
        "not statistically significant rating" (կամ nssr)՝ հակառակ դեպքում։
        """
        
        if self.num_users_rated > 5000:
            return 'SSR'
        else:
            return 'NSSR'

In [5]:
class Library:
    
    def __init__(self):
        """
        Create an attribute of type list called Book_list which will be an empty list for now.
        
        Ստեղծեք list տիպի Book_list attribute-ը, որն առայժմ կլինի դատարկ list 
        """
        
        self.book_list = []
    
    def get_top_5(self):
        """
        Return the information related to 5 books that have the highest rating using the method pretty_print.
        If there are more than 5 books with the same rating, randomly select and show only 5 of them.
        
        Վերադարձրեք 5 ամենաբարձր գնահատական ունեցող գրքերի վերաբերյալ ինֆորմացիա՝ օգտվելով pretty_print
        ֆունկցիայից։ Եթե այդպիսի գրքերը 5-ից ավելի են՝ պատահականության սկզբունքով ընտրեք դրանցից 5-ը։
        """
        
        new_list = sorted(self.book_list, key=lambda x: x.rating, reverse=True)
        new_list_2 = new_list[:5]
        
        
        i = 4
        while i < len(new_list) - 1:
            if new_list[i].rating == new_list[i+1].rating:
                new_list_2.append(new_list[i+1])
                i += 1
            else:
                break
            
        index = np.random.choice(range(len(new_list_2)), 5, replace=False)  
        
        for i in index:
            new_list_2[i].pretty_print()
            print('\n\n')
            
    
    def simple_search(self, title):
        """
        Search a book with required title and print information related to all books with titles exactly 
        matching the searched title using the method pretty_print.
        
        Search արեք տրված վերնագրով գիրք (գրքեր) և վերադարձրեք այդ գրքի (գրքերի) մասին ինֆորմացիան՝ օգտվելով 
        pretty_print ֆունկցիայից։ Վերադարձված գրքերի վերնագրերը պետք է համընկնեն title փոփոխականի արժեքի հետ
        """
        
        for book in self.book_list:
            if book.title == title:
                book.pretty_print()
    
    def complex_search(self, title):
        """
        In this case the user may not remember the exact title of the book. If you have a book(s) with 
        title exactly matching the searched title, this method should perform just like the method simple_search.
        Otherwise, if you don't have any books with the searched title, you should return book(s) that have the
        most similar titles. You are free to implement this method however you like. 
        One version that you can implement in case you don't have other ideas:
        return the book(s) with titles containing the most number of words from the searched title 
        (return the information using the method pretty_print), and if there are no such books, 
        return "nothing macthed your search".
        I would be happy to see any other interesting versions of the method :)
        
        
        Այս դեպքում օգտատերը միգուցե չհիշի գրքի կոնկրետ անվանումը կամ այն մի փոքր սխալ հիշի։ Եթե կա գիրք (գրքեր)
        որոնց վերնագիրը համընկնում է title փոփոխականի արժեքի հետ, ապա վերադարձրեք այդ գրքին (գրքերին) 
        վերաբերող ինֆորմացիան՝ օգտվելով pretty_print ֆունկցիայից։ Հակառակ դեպքում, պետք է վերադարձնել այն գրքերի մասին 
        ինֆորմացիա, որոնց վերնագրերը նման են title փոփոխականում պահված վերնագրի արժեքին։ Ազատ եք
        որոշելու այս ֆունկցիայի աշխատանքի տրամաբանությունը։
        Որպես հնարավոր տարբերակ (եթե չկան այլ մտքեր) կարող եք վերադարձնել այն գրքերի ինֆորմացին, որոնց
        վերնագրերը պարունակում են title փոփոխամանում տրված բառերից ամենաշատ քանակության բառերը ու եթե չկան
        այդպիսի գրքեր, վերադարձրեք "nothing macthed your search"։ 
        Ուրախ կլինեմ տեսնել այս ֆունկցիայի հետաքրքիր տարբերակներ :)
        """
        
        for book in self.book_list:
            if book.title == title:
                book.pretty_print()
                return
        
        distances = {}
        
        words_input = title.lower().split(' ')
        all_scores = {}
        
        for book in self.book_list:
            title_book = book.title
            words_book_title = title_book.lower().split(' ')
            
            score = 0
            for word1 in words_input:
                scores = []
                for word2 in words_book_title:
                    scores.append(levenshtein_distance(word1,word2))
                score += min(scores)
            all_scores[title_book] = score + abs(len(words_input) - len(words_book_title))
            
        all_scores = dict(sorted(all_scores.items(), key=lambda item: item[1]))
        min_score = list(all_scores.values())[0]
        min_score_title = list(all_scores.keys())[0]
        
        threshold = len(min_score_title.split(' '))
        if(min_score >= threshold):
            print('nothing matched with your search')
        else:
            return self.simple_search(min_score_title)
        
        

In [6]:
books_list = []
books = pd.read_csv('data.csv')

for i in range(len(books)):
    row = books.iloc[i]
    title = row['Name']
    price = row['Price']
    author = row['Author']
    rating = row['Rating']
    count = row['Users']
    
    book = Book(title, author, rating, count, price)
    books_list.append(book)
    
library = Library()
library.book_list = books_list

In [7]:
library.simple_search('I Love You to the Moon and Back')

I Love You to the Moon and Back by Amelia Hepworth with rating 4.9 costs $3.59


In [189]:
library.complex_search('lve ou to and bac')
library.complex_search('jg owegfh we')
library.complex_search('world travel: guide')
library.complex_search('wrd trv')

I Love You to the Moon and Back by Amelia Hepworth with rating 4.9 costs $3.59
nothing matched with your search
World Travel: An Irreverent Guide by Anthony Bourdain with rating 4.8 costs $22.8
nothing matched with your search


## Task 3.

```
Write a short summary explaining what you have implemented in the 2 Tasks. Don't go too much into details since I can also see your code. Just incorporate some points about interesting decisions or solutions that you have applied. 
```

```
Գրեք Ձեր աշխատանքի փոքրիկ ամփոփում, որտեղ կբացատրեք թե ինչպես եք կատարել առաջադրանքները։ Ամփոփումը պետք է լինի շատ կարճ ու ներառի միայն այս կամ այն լուծում տալու վերաբերյալ որոշ բացատրություններ։ Հարկավոր չէ բացատրել ամեն մի փոքր քայլ։ 
```

## Scrapping is done by getting the requested page data and searching needed elements in that data.

## Class functions were simple. 
## The complex search 

```
Complex search is implemented with taking into consideration levenshtein distance of words. We can get scores (distances) for 2 titles with comparing the levenshtein distance of words in that titles. If overall score is bigger than the given threshold than we don't find any titles. Otherwise we will return the minimum score title
```